In [ ]:
import tensorflow as tf
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
import os, os.path
from pathlib import Path

# Setting for the Training run


In [ ]:
# Parameters to set for the run
image_size = 128
train_ds_link = "path_to_train_dataset"
test_ds_link = "path_to_test_dataset"
validation_split = 0.1
color_mode = "grayscale"
learning_rate=0.005
name_of_run = "name_of_your_run"
epochs = 25
batch_size = 32

reduce_lr = True


# Import the Data

In [ ]:
print("Start reading the Training Dataset...")

imdg = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=validation_split)#, vertical_flip=True, rotation_range=20) 

train_ds = tf.keras.preprocessing.image.DirectoryIterator(
    train_ds_link,
    #labels="inferred",
    image_data_generator=imdg,
    class_mode="categorical",
    #class_names=None,
    color_mode=color_mode,
    batch_size=batch_size,
    target_size=(image_size,image_size),
    shuffle=True,
    seed=123,
    interpolation="bilinear",
    follow_links=False,
    #validation_split=0.2,
    subset="training"
)
print("Start reading the Validation Dataset...")

validation_ds = tf.keras.preprocessing.image.DirectoryIterator(
    train_ds_link,
    #labels="inferred",
    image_data_generator=imdg,
    class_mode="categorical",
    #class_names=None,
    color_mode=color_mode,
    batch_size=batch_size,
    target_size=(image_size,image_size),
    shuffle=True,
    seed=123,
    interpolation="bilinear",
    follow_links=False,
    #validation_split=0.2,
    subset="validation"
)

print("Finished reading the Datasets!")


# Calculate Class Weights

In [ ]:
def get_n_per_class():
    n = []
    for i in range(6):
        DIR = train_ds_link + str(i+1) + "/"
        x = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR,name))])
        n.append(x)
    return n

n = get_n_per_class()
print(n)

In [ ]:
# weight_class1 = number_samples_in_dataset /(number_of_classes * number_samples_class1)
def class_weights(n):
    number_samples_in_dataset = sum(n)
    print("Number of total samples: " + str(number_samples_in_dataset))
    class_w = []
    number_of_classes = 6
    labels = [0,1,2,3,4,5]
    for i in n:
        print(i)
        class_w.append(number_samples_in_dataset / (number_of_classes * i))
    return dict(zip(labels,class_w))

class_w = class_weights(n)
print(class_w)

# Create the Model

In [ ]:
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3,3), padding= 'same', input_shape= (image_size, image_size, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (5,5), padding= 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3,3), padding= 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3,3), padding= 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(6, activation='softmax'))

opt = Adam(lr=learning_rate)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics= ['accuracy'])
model.summary()


# Train the Model

In [ ]:
try_name = name_of_run
Path("./" + try_name).mkdir(parents=True, exist_ok=True)

checkpoint = ModelCheckpoint(try_name + "model_weights.h5", monitor='val_accuracy', save_weights_only=True, mode='max' , verbose=1 )
callbacks = [checkpoint]
if(reduce_lr):
    reduce_lr_ = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience = 2, min_lr = 0.00001, model = 'auto')
    callbacks.append(reduce_lr_)

history = model.fit(train_ds, validation_data=validation_ds,epochs=epochs, verbose=1,callbacks=callbacks,class_weight=class_w)

In [ ]:
# Save the Model
model_json = model.to_json()
with open(try_name + "model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(try_name + "end_model.h5")
print("Saved model to disk")

# Plot training metrics

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.savefig(try_name + 'acc.png')

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.savefig(try_name + 'loss.png')

# Evaluate the model

In [ ]:
imdg = tf.keras.preprocessing.image.ImageDataGenerator()

test_ds = tf.keras.preprocessing.image.DirectoryIterator(
    test_ds_link,
    #labels="inferred",
    image_data_generator=imdg,
    class_mode="categorical",
    #class_names=None,
    color_mode=color_mode,
    batch_size=batch_size,
    target_size=(image_size,image_size),
    shuffle=False,
    seed=123,
    interpolation="bilinear",
    follow_links=False,
)

## Testing loss and accuracy

In [ ]:
evaluation = model.evaluate(test_ds, verbose=1)
print(evaluation)

## Calculate confusion matrix, precision, recall and f1-score

In [ ]:
# model.load_weights('P003_128x128/model_weights.h5')
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(test_ds, verbose=1)

y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm = confusion_matrix(test_ds.classes, y_pred)
print(cm)
print('Classification Report')
target_names = ["Happiness", "Sadness", "Surprise", "Fear", "Disgust", "Anger"]
print(classification_report(test_ds.classes, y_pred, target_names=target_names))

### Plot confusion matrix with absolute values

In [ ]:
import seaborn as sn
import pandas as pd

sn.set(font_scale=1.4)
plt.figure(figsize = (10,7))
sn.heatmap(cm, xticklabels=target_names, yticklabels=target_names, annot=True, annot_kws={"size": 14},fmt="d")
plt.ylabel("True label")
plt.xlabel("Predicted label")

### Plot confusion matrix with percentage values

In [ ]:
import seaborn as sn
import pandas as pd

sn.set(font_scale=1.4)
plt.figure(figsize = (10,7))
sn.heatmap(cm/np.sum(cm), xticklabels=target_names, yticklabels=target_names, annot=True, annot_kws={"size": 14},fmt=".2%")
plt.ylabel("True label")
plt.xlabel("Predicted label")